# Final version

In [39]:
import numpy as np
import random
import pandas as pd
import heapq
import time  # For measuring computation time

# Environment related code

In [45]:
class DisasterZoneEnv:
    def __init__(self, width=8, height=8, num_obstacles=5, num_survivors=3, num_resources=2, initial_energy=20, dynamic=False):
        self.width = width
        self.height = height
        self.num_obstacles = num_obstacles
        self.num_survivors = num_survivors
        self.num_resources = num_resources
        self.initial_energy = initial_energy
        self.dynamic = dynamic
        self.reset()

    def reset(self):
        self.grid = np.zeros((self.height, self.width), dtype=int)
        for _ in range(self.num_obstacles):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 1
        for _ in range(self.num_survivors):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 2
        for _ in range(self.num_resources):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 3
        self.drone_x, self.drone_y = self._get_random_empty_cell()
        self.energy = self.initial_energy
        return self._get_state()

    def _get_random_empty_cell(self):
        while True:
            x = random.randint(0, self.height - 1)
            y = random.randint(0, self.width - 1)
            if self.grid[x, y] == 0:
                return x, y

    def _get_state(self):
        return (self.drone_x, self.drone_y, self.energy)

    def step(self, new_x, new_y):
        reward = 0
        done = False

        if not self._in_bounds(new_x, new_y):
            reward -= 10
        elif self.grid[new_x, new_y] == 1:
            reward -= 10
        else:
            self.drone_x, self.drone_y = new_x, new_y
            if self.grid[new_x, new_y] == 0:
                reward += 1
            elif self.grid[new_x, new_y] == 2:
                reward += 10
                self.grid[new_x, new_y] = 0
            elif self.grid[new_x, new_y] == 3:
                reward += 5
                self.energy += 5
                self.grid[new_x, new_y] = 0

        self.energy -= 1
        reward -= 1
        if self.energy <= 0:
            done = True

        return self._get_state(), reward, done

    def _in_bounds(self, x, y):
        return 0 <= x < self.height and 0 <= y < self.width

    def render(self):
        grid_copy = self.grid.astype(str)
        grid_copy[grid_copy == '0'] = '.'
        grid_copy[grid_copy == '1'] = '#'
        grid_copy[grid_copy == '2'] = 'S'
        grid_copy[grid_copy == '3'] = 'R'
        grid_copy[self.drone_x, self.drone_y] = 'D'
        for row in grid_copy:
            print(" ".join(row))
        print(f"Energy: {self.energy}\n")




# Dijiksra Agent

In [46]:
class DijkstraAgent:
    def __init__(self, env):
        self.env = env
        self.steps_taken = 0  # Initialize steps counter

    def dijkstra(self, grid, start, target):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        pq = [(0, start)]
        distances = {start: 0}
        previous = {start: None}
        while pq:
            current_distance, current_position = heapq.heappop(pq)
            if current_position == target:
                path = []
                while current_position:
                    path.append(current_position)
                    current_position = previous[current_position]
                path.reverse()
                return path, current_distance
            for dx, dy in directions:
                neighbor = (current_position[0] + dx, current_position[1] + dy)
                if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1]:
                    if grid[neighbor[0], neighbor[1]] != 1:
                        new_distance = current_distance + 1
                        if neighbor not in distances or new_distance < distances[neighbor]:
                            distances[neighbor] = new_distance
                            previous[neighbor] = current_position
                            heapq.heappush(pq, (new_distance, neighbor))
        return [], float('inf')

    def find_closest_target(self, target_type):
        start = (self.env.drone_x, self.env.drone_y)
        targets = [(x, y) for x in range(self.env.height) for y in range(self.env.width) if self.env.grid[x, y] == target_type]
        if not targets:
            return None, None, float('inf')
        shortest_path, closest_target, shortest_distance = None, None, float('inf')
        for target in targets:
            path, distance = self.dijkstra(self.env.grid, start, target)
            if distance < shortest_distance:
                shortest_path, closest_target, shortest_distance = path, target, distance
        return shortest_path, closest_target, shortest_distance

    def execute(self):
        while self.env.energy > 0:
            path, target_position, distance = self.find_closest_target(2)
            if not path:
                path, target_position, distance = self.find_closest_target(3)
            if not path:
                print("No reachable targets remaining. Stopping simulation.")
                break
            for step in path[1:]:
                self.env.step(*step)
                self.steps_taken += 1  # Increment steps
                if self.env.energy <= 0:
                    print("Drone ran out of energy!")
                    break
            if target_position:
                self.env.grid[target_position[0], target_position[1]] = 0
        print("Agent has completed its task.")


# Tester

In [47]:
class Tester:
    def __init__(self, env, num_tests=5):
        """
        Initialize the tester.

        :param env: The DisasterZoneEnv instance.
        :param num_tests: Number of test runs to perform.
        """
        self.env = env
        self.num_tests = num_tests
        self.results = []

    def run(self, agent_class):
        """
        Run the tests for the specified agent class.

        :param agent_class: The agent class to test (e.g., DijkstraAgent).
        """
        for test_idx in range(self.num_tests):
            self.env.reset()
            agent = agent_class(self.env)

            # Record initial metrics
            survivors_before = np.sum(self.env.grid == 2)
            resources_before = np.sum(self.env.grid == 3)
            energy_before = self.env.energy

            # Measure computation time
            start_time = time.time()
            agent.execute()
            end_time = time.time()

            # Record final metrics
            survivors_after = np.sum(self.env.grid == 2)
            resources_after = np.sum(self.env.grid == 3)
            energy_after = self.env.energy

            # Append results for this test
            self.results.append({
                "Test": test_idx + 1,
                "Grid Dimensions": f"{self.env.width}x{self.env.height}",
                "Obstacles": self.env.num_obstacles,
                "Survivors Rescued": survivors_before - survivors_after,
                "Resources Collected": resources_before - resources_after,
                "Energy Used": energy_before - energy_after,
                "Steps Taken": agent.steps_taken,
                "Computation Time (s)": round(end_time - start_time, 4),
            })

    def display_results(self):
        """
        Display the collected test results in a table format.
        """
        if not self.results:
            print("No results to display.")
            return

        df = pd.DataFrame(self.results)
        print("\nResults Table:")
        print(df)

    def save_results_to_csv(self, filename="test_results.csv"):
        """
        Save the test results to a CSV file.

        :param filename: Name of the CSV file to save the results (default: 'test_results.csv').
        """
        if not self.results:
            print("No results to save.")
            return

        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")


# Code

In [48]:
if __name__ == "__main__":
    env = DisasterZoneEnv(width=8, height=8, num_obstacles=5, num_survivors=3, num_resources=2, initial_energy=20, dynamic=False)
    tester = Tester(env, num_tests=3)

    # Run the tests
    tester.run(agent_class=DijkstraAgent)

    # Display results
    tester.display_results()

    # Save results to a CSV file
    tester.save_results_to_csv("dijkstra_test_results.csv")


Drone ran out of energy!
Agent has completed its task.
No reachable targets remaining. Stopping simulation.
Agent has completed its task.
No reachable targets remaining. Stopping simulation.
Agent has completed its task.

Results Table:
   Test Grid Dimensions  Obstacles  Survivors Rescued  Resources Collected  \
0     1             8x8          5                  3                    1   
1     2             8x8          5                  3                    2   
2     3             8x8          5                  3                    2   

   Energy Used  Steps Taken  Computation Time (s)  
0           20           20                0.0015  
1           13           23                0.0012  
2           12           22                0.0011  
Results saved to dijkstra_test_results.csv
